In [4]:
import sys
#!{sys.executable} -m pip install ccxt pandas
import ccxt
import pandas as pd

In [5]:
exchange = ccxt.binance({
    'apiKey': 'YOUR_API_KEY',
    'secret': 'YOUR_SECRET_KEY',
    'enableRateLimit': True,
})

In [6]:
def fetch_ohlcv(symbol, timeframe):
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df

In [7]:
def calculate_moving_averages(df, short_window, long_window):
    df['short_mavg'] = df['close'].rolling(window=short_window).mean()
    df['long_mavg'] = df['close'].rolling(window=long_window).mean()
    return df

In [8]:
def execute_orders(df):
    in_position = False

    for index, row in df.iterrows():
        if row['short_mavg'] > row['long_mavg'] and not in_position:
            print("Buy")
            # exchange.create_market_buy_order('BTC/USDT', amount)
            in_position = True
        elif row['short_mavg'] < row['long_mavg'] and in_position:
            print("Sell")
            # exchange.create_market_sell_order('BTC/USDT', amount)
            in_position = False

In [9]:
symbol = 'BTC/USDT'
timeframe = '1h'
short_window = 50
long_window = 100

df = fetch_ohlcv(symbol, timeframe)
df = calculate_moving_averages(df, short_window, long_window)
execute_orders(df)

AuthenticationError: binance {"code":-2008,"msg":"Invalid Api-Key ID."}